In [1]:
!pip install transformers datasets rouge_score

from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from rouge_score import rouge_scorer
import numpy as np

# 1. Load a VERY Small Subset of the Dataset
try:
    dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:1%]")  # Use only 1% of the training data
except ValueError as e:
    print(f"Error loading dataset: {e}")
    print("Check dataset name, version, and Hugging Face Hub availability.")
    raise

# 2. Choose a Smaller Model
model_name = 't5-small'  # 't5-small' is much faster than 't5-base'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# 3. Preprocess Data (Simplified)
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding="max_length", return_tensors="pt")  # Shorter max length

    labels = tokenizer(examples["highlights"], max_length=64, truncation=True, padding="max_length", return_tensors="pt") # Shorter max length

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

# 4. Training (Fewer Epochs, Smaller Batch Size)
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",  # Skip evaluation during training for speed
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Increase batch size if GPU allows
    per_device_eval_batch_size=8,
    num_train_epochs=1,  # Train for only ONE epoch
    weight_decay=0.01,
    save_steps=10000, #avoid saving checkpoints
    logging_steps=100,
    push_to_hub=False,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    #eval_dataset=tokenized_datasets, #remove eval dataset
    tokenizer=tokenizer,
)

trainer.train()

# 5. Simplified Evaluation (Optional)
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = []

    for ref, pred in zip(decoded_labels, decoded_preds):
        try:
            score = scorer.score(ref, pred)
            rouge_scores.append(score)
        except Exception as e:
            print(f"Error calculating ROUGE score: {e}")
            print(f"Reference: {ref}")
            print(f"Prediction: {pred}")
            # Handle the error, e.g., by skipping this pair or assigning a default score
            continue

    # Calculate average ROUGE scores, handling cases where rouge_scores is empty
    if rouge_scores:
        avg_rouge1 = np.mean([score['rouge1'].fmeasure for score in rouge_scores])
        avg_rouge2 = np.mean([score['rouge2'].fmeasure for score in rouge_scores])
        avg_rougeL = np.mean([score['rougeL'].fmeasure for score in rouge_scores])

        return {
            'rouge1': avg_rouge1,
            'rouge2': avg_rouge2,
            'rougeL': avg_rougeL,
        }
    else:
        return {
            'rouge1': 0.0,
            'rouge2': 0.0,
            'rougeL': 0.0,
            'error': "No valid ROUGE scores calculated"
        }

# Override the prediction_step in Trainer to handle labels correctly
class CustomTrainer(Trainer):
    def prediction_step(
        self,
        model,
        inputs,
        prediction_loss_only,
        ignore_keys = None
    ):
        labels = inputs["labels"].clone()
        labels[labels == tokenizer.pad_token_id] = -100
        inputs["labels"] = labels
        return super().prediction_step(
            model, inputs, prediction_loss_only=prediction_loss_only, ignore_keys=ignore_keys
        )
# comment out to skip evaluation

#trainer = CustomTrainer(
#    model=model,
#    args=training_args,
#    train_dataset=tokenized_datasets,
#    eval_dataset=tokenized_datasets,
#    tokenizer=tokenizer,
#    compute_metrics=compute_metrics,  # Pass the compute_metrics function here
#)

#trainer.train()
#eval_results = trainer.evaluate()
#print(f"Evaluation Results: {eval_results}")

# 6. Inference
text = "Climate change is increasingly affecting global agriculture, posing significant challenges to food security and livelihoods. Rising temperatures, altered precipitation patterns, and increased frequency of extreme weather events are some of the key factors impacting crop yields and livestock productivity. For instance, prolonged droughts can lead to water scarcity, affecting irrigation and reducing crop growth. Similarly, excessive rainfall can cause flooding, damaging crops and soil health."
input_ids = tokenizer.encode("summarize: " + text, return_tensors="pt").to(model.device)
output = model.generate(input_ids, max_length=100, num_beams=4, early_stopping=True)  # Shorter summary length
summary = tokenizer.decode(output[0], skip_special_tokens=True)
print(summary)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.6 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=19c3aa870b90cb0b9b39371ec4b4cb5db6a4fa86361c23666298d6d64d142bab
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Map:   0%|          | 0/2871 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-1-35bbd2eafdc0>:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
100,3.380700
200,2.629800
300,2.507100


Rising temperatures, altered precipitation patterns, increased frequency of extreme weather events are some of the key factors impacting crop yields and livestock. Similarly, excessive rainfall can cause flooding, damaging crops and soil health.
